In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
extracted_features_save_adr = "./extracted_features.pickle"

In [ ]:
# read extracted features from pickle file
def read_extracted_features(extracted_features_save_adr):
    with open(extracted_features_save_adr, 'rb') as input_file:
        return pickle.load(input_file)

In [ ]:
features = read_extracted_features(extracted_features_save_adr)
features = dict(features)
averaged_features = {}

In [ ]:
df = pd.DataFrame.from_dict(features, orient='index')
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'patient_id'})
df['patient_id'] = df['patient_id'].astype(np.int64)
df.to_csv('features_for_all_patches.csv', index=False, header=True)

In [ ]:
# Create a column with the weight of each patient based on the number of patches
weight_col = []
total_weight = 0
for patient in features.keys():
    weight_col.append(len(features[patient]))
    total_weight += len(features[patient])
    total_weight = float(total_weight)
weight_col = np.array(weight_col, dtype=np.float32)
weight_col = weight_col / total_weight

In [ ]:
# keep the median for each patient patches
for patient in features.keys():
    averaged_features.update({patient: np.median(list(features[patient].values()), axis=0)})

In [ ]:
# Convert the dictionary to a csv file where each row is a patient and each column is a feature using pandas
df = pd.DataFrame.from_dict(averaged_features, orient='index')
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'patient_id'})
df['patient_id'] = df['patient_id'].astype(np.int64)
#df['weight'] = weight_col
df.to_csv('features.csv', index=False, header=True)

In [ ]:
# Merge the extracted features with the clinical data on the patient ID
clinical_data = pd.read_csv('clinical_data_with_no_missing_values.csv')
clinical_data['patient_id'] = clinical_data['patient_id'].astype(np.int64)
features_with_clinical_data = clinical_data.merge(df, on='patient_id')
features_with_clinical_data.to_csv('features_with_clinical_data_1024_2.csv', index=False, header=True)

In [ ]:
df = pd.read_csv('features_with_clinical_data_1024_2.csv')
df